## Identifying handwritten digits using Logistic Regression in PyTorch

### Import Libraries

In [1]:
import torch 
import torch.nn as nn 
import torchvision.datasets as dsets 
import torchvision.transforms as transforms 
from torch.autograd import Variable

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'gpu'
device

'cuda'

### Reading the Data

In [3]:
# Hyper Parameters  
input_size = 784 # Image size is 28x28
num_classes = 10 # 10 digits
num_epochs = 5
batch_size = 100 # prevent memory overflow
learning_rate = 0.001
path = r'../dataset'

In [4]:
# MNIST Dataset (Images and Labels) 
train_dataset = dsets.MNIST(root = path,  
                            train = True,  
                            transform = transforms.ToTensor(), 
                            download = True) 
  
test_dataset = dsets.MNIST(root = path,  
                           train = False,  
                           transform = transforms.ToTensor()) 
  
# Dataset Loader (Input Pipeline) 
train_loader = torch.utils.data.DataLoader(dataset = train_dataset,  
                                           batch_size = batch_size,  
                                           shuffle = True) 
  
test_loader = torch.utils.data.DataLoader(dataset = test_dataset,  
                                          batch_size = batch_size,  
                                          shuffle = False)

### Build Model

In [5]:
class LogisticRegression(nn.Module): 
    def __init__(self, input_size, num_classes): 
        super().__init__() 
        self.linear = nn.Linear(input_size, num_classes)
  
    def forward(self, x): 
        out = self.linear(x) # the softmax is internally calculated during each forward pass and hence we do not need to specify it inside the forward() function
        return out 

In [15]:
class ConvNet(nn.Module):
    def __init__(self, num_classes=10):
        super(ConvNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.fc = nn.Linear(7*7*32, num_classes)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)  # Flatten the tensor
        out = self.fc(out)
        return out

In [7]:
model1 = LogisticRegression(input_size, num_classes).to(device)

In [17]:
model2 = ConvNet(num_classes).to(device)

### Loss Function and Optimizer

In [18]:
loss_fn = nn.CrossEntropyLoss() 
optimizer = torch.optim.SGD(model2.parameters(), lr = learning_rate) 

### Training

1. Reset all gradients to 0.
2. Make a forward pass.
3. Calculate the loss.
4. Perform backpropagation.
5. Update all weights.

In [21]:
# Training the Model
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # Reshape images to (batch_size, input_size)
        # images = images.view(-1, 28*28)  # Flatten the images
        
        # Move images and labels to the appropriate device (e.g., GPU if available)
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass: compute the model output
        outputs = model2(images)
        
        # Compute the loss
        loss = loss_fn(outputs, labels)
        
        # Zero the gradients before running the backward pass
        optimizer.zero_grad()
        
        # Backward pass: compute gradient of the loss with respect to model parameters
        loss.backward()
        
        # Perform a single optimization step (parameter update)
        optimizer.step()
        
        # Print statistics
        if (i + 1) % 100 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'.format(
                epoch + 1, num_epochs, i + 1, len(train_dataset) // batch_size, loss.item()))

Epoch [1/5], Step [100/600], Loss: 1.2331
Epoch [1/5], Step [200/600], Loss: 0.8276
Epoch [1/5], Step [300/600], Loss: 0.6686
Epoch [1/5], Step [400/600], Loss: 0.5136
Epoch [1/5], Step [500/600], Loss: 0.5889
Epoch [1/5], Step [600/600], Loss: 0.4266
Epoch [2/5], Step [100/600], Loss: 0.3687
Epoch [2/5], Step [200/600], Loss: 0.3629
Epoch [2/5], Step [300/600], Loss: 0.4064
Epoch [2/5], Step [400/600], Loss: 0.2378
Epoch [2/5], Step [500/600], Loss: 0.3001
Epoch [2/5], Step [600/600], Loss: 0.2727
Epoch [3/5], Step [100/600], Loss: 0.2839
Epoch [3/5], Step [200/600], Loss: 0.2595
Epoch [3/5], Step [300/600], Loss: 0.2963
Epoch [3/5], Step [400/600], Loss: 0.2010
Epoch [3/5], Step [500/600], Loss: 0.1330
Epoch [3/5], Step [600/600], Loss: 0.3175
Epoch [4/5], Step [100/600], Loss: 0.2267
Epoch [4/5], Step [200/600], Loss: 0.2116
Epoch [4/5], Step [300/600], Loss: 0.2220
Epoch [4/5], Step [400/600], Loss: 0.2190
Epoch [4/5], Step [500/600], Loss: 0.3008
Epoch [4/5], Step [600/600], Loss:

### Testing the model

In [24]:
# Test the Model
correct = 0
total = 0
for images, labels in test_loader:
    
    # Move images and labels to the appropriate device (e.g., GPU if available)
    images = images.to(device)
    labels = labels.to(device)
    
    # Reshape and predict
    # images = images.view(-1, 28 * 28)
    outputs = model2(images)
    
    # Get predictions from the maximum value
    _, predicted = torch.max(outputs, 1)
    
    # Total number of labels
    total += labels.size(0)
    
    # Total correct predictions
    correct += (predicted == labels).sum().item()

# Calculate accuracy
accuracy = 100 * correct / total
print('Accuracy of the model on the 10000 test images: %d %%' % accuracy)

Accuracy of the model on the 10000 test images: 96 %
